# Лабораторная работа №5  
## Проведение исследований с градиентным бустингом (Gradient Boosting)

В лабораторной работе исследуются модели градиентного бустинга:
- **GradientBoostingClassifier** для задачи классификации (loan_status);
- **GradientBoostingRegressor** для задачи регрессии (song_popularity).

## Импорт библиотек

In [5]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy import sparse
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    root_mean_squared_error, mean_absolute_error, r2_score
)

RANDOM_STATE = 42

## 1. Загрузка данных и формирование выборок

- отделяется целевая переменная;
- удаляются идентификаторы (`customer_id`, `song_name`);
- выделяются категориальные и числовые признаки;
- выполняется разбиение train/test.

In [2]:
# Пути к данным
LOAN_PATH = os.path.join("data", "Loan_approval_data_2025.csv")
SONG_PATH = os.path.join("data", "song_data.csv")

# Загрузка датасетов
loan_df = pd.read_csv(LOAN_PATH)
song_df = pd.read_csv(SONG_PATH)

# --- Classification ---
y_cls = loan_df["loan_status"].astype(int)
X_cls = loan_df.drop(columns=["loan_status"])

if "customer_id" in X_cls.columns:
    X_cls = X_cls.drop(columns=["customer_id"])

cat_cols_cls = X_cls.select_dtypes(include=["object"]).columns.tolist()
num_cols_cls = X_cls.select_dtypes(exclude=["object"]).columns.tolist()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls
)

# --- Regression ---
y_reg = song_df["song_popularity"].astype(float)
X_reg = song_df.drop(columns=["song_popularity"])

if "song_name" in X_reg.columns:
    X_reg = X_reg.drop(columns=["song_name"])

cat_cols_reg = X_reg.select_dtypes(include=["object"]).columns.tolist()
num_cols_reg = X_reg.select_dtypes(exclude=["object"]).columns.tolist()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print("Classification:", Xc_train.shape, Xc_test.shape)
print("Regression:", Xr_train.shape, Xr_test.shape)

Classification: (40000, 18) (10000, 18)
Regression: (15068, 13) (3767, 13)


## 2. Бейзлайн и оценка качества (Gradient Boosting)

Построение бейзлайна:
- GradientBoostingClassifier;
- GradientBoostingRegressor.

Алгоритм работает с числовыми признаками, поэтому категориальные кодируются через one-hot.
Масштабирование не обязательно для деревьев, поэтому не используется.

Затем рассчитываются метрики:
- классификация: Accuracy, Precision, Recall, F1, ROC-AUC;
- регрессия: RMSE, MAE, R².

In [3]:
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

prep_cls = ColumnTransformer([
    ("num", num_pipe, num_cols_cls),
    ("cat", cat_pipe, cat_cols_cls)
])

prep_reg = ColumnTransformer([
    ("num", num_pipe, num_cols_reg),
    ("cat", cat_pipe, cat_cols_reg)
])

baseline_cls = Pipeline([
    ("prep", prep_cls),
    ("model", GradientBoostingClassifier(random_state=RANDOM_STATE))
])

baseline_reg = Pipeline([
    ("prep", prep_reg),
    ("model", GradientBoostingRegressor(random_state=RANDOM_STATE))
])

baseline_cls.fit(Xc_train, yc_train)
baseline_reg.fit(Xr_train, yr_train)

yc_pred = baseline_cls.predict(Xc_test)
yc_proba = baseline_cls.predict_proba(Xc_test)[:, 1]

yr_pred = baseline_reg.predict(Xr_test)

baseline_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred),
    "precision": precision_score(yc_test, yc_pred, zero_division=0),
    "recall": recall_score(yc_test, yc_pred, zero_division=0),
    "f1": f1_score(yc_test, yc_pred, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba)
}

baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred),
    "mae": mean_absolute_error(yr_test, yr_pred),
    "r2": r2_score(yr_test, yr_pred)
}

print("Baseline classification:", baseline_cls_res)
print("Baseline regression:", baseline_reg_res)

Baseline classification: {'accuracy': 0.9194, 'precision': 0.913135220678741, 'recall': 0.943324250681199, 'f1': 0.9279842744817727, 'roc_auc': np.float64(0.9797343298993028)}
Baseline regression: {'rmse': 20.705977881928618, 'mae': 16.41457783620347, 'r2': 0.11064459479883026}


## 3. Улучшение бейзлайна: гипотезы и проверка

### Гипотезы для классификации
1) Подбор `n_estimators`, `learning_rate`, `max_depth` (через `max_depth` базового дерева) улучшит ROC-AUC/F1.  
2) Подбор `subsample` (стохастический бустинг) может снизить переобучение.

### Гипотезы для регрессии
1) Аналогичный подбор параметров уменьшит RMSE.  
2) Ограничение глубины деревьев (через `max_depth`) улучшит обобщающую способность.

Ниже проводится GridSearchCV:
- классификация оптимизируется по ROC-AUC;
- регрессия оптимизируется по neg-RMSE.

In [6]:

# --- Tuning: Classification (RandomizedSearch) ---
param_dist_cls = {
    "model__n_estimators": randint(80, 260),          # было [100,200,400]
    "model__learning_rate": uniform(0.03, 0.17),      # примерно 0.03..0.20
    "model__max_depth": randint(2, 5),                # 2..4
    "model__subsample": uniform(0.75, 0.25),          # 0.75..1.0
}

tuned_cls = Pipeline([
    ("prep", prep_cls),
    ("model", GradientBoostingClassifier(
        random_state=RANDOM_STATE,
        n_iter_no_change=10,   # ранняя остановка
        validation_fraction=0.1,
        tol=1e-4
    ))
])

cv_cls = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

rs_cls = RandomizedSearchCV(
    tuned_cls,
    param_distributions=param_dist_cls,
    n_iter=20,                # 20 вместо полного перебора 54
    cv=cv_cls,
    scoring="roc_auc",
    random_state=RANDOM_STATE,
    n_jobs=1,
    verbose=1
)

rs_cls.fit(Xc_train, yc_train)

best_cls = rs_cls.best_estimator_
print("Best cls params:", rs_cls.best_params_)
print("Best CV ROC-AUC:", rs_cls.best_score_)

yc_pred2 = best_cls.predict(Xc_test)
yc_proba2 = best_cls.predict_proba(Xc_test)[:, 1]

improved_cls_res = {
    "accuracy": accuracy_score(yc_test, yc_pred2),
    "precision": precision_score(yc_test, yc_pred2, zero_division=0),
    "recall": recall_score(yc_test, yc_pred2, zero_division=0),
    "f1": f1_score(yc_test, yc_pred2, zero_division=0),
    "roc_auc": roc_auc_score(yc_test, yc_proba2)
}
print("Improved classification:", improved_cls_res)


# --- Tuning: Regression (RandomizedSearch) ---
param_dist_reg = {
    "model__n_estimators": randint(80, 260),
    "model__learning_rate": uniform(0.03, 0.17),
    "model__max_depth": randint(2, 5),
    "model__subsample": uniform(0.75, 0.25),
}

tuned_reg = Pipeline([
    ("prep", prep_reg),
    ("model", GradientBoostingRegressor(
        random_state=RANDOM_STATE,
        n_iter_no_change=10,   # ранняя остановка
        validation_fraction=0.1,
        tol=1e-4
    ))
])

cv_reg = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

rs_reg = RandomizedSearchCV(
    tuned_reg,
    param_distributions=param_dist_reg,
    n_iter=20,
    cv=cv_reg,
    scoring="neg_root_mean_squared_error",
    random_state=RANDOM_STATE,
    n_jobs=1,
    verbose=1
)

rs_reg.fit(Xr_train, yr_train)

best_reg = rs_reg.best_estimator_
print("Best reg params:", rs_reg.best_params_)
print("Best CV -RMSE:", rs_reg.best_score_)

yr_pred2 = best_reg.predict(Xr_test)

improved_reg_res = {
    "rmse": root_mean_squared_error(yr_test, yr_pred2),
    "mae": mean_absolute_error(yr_test, yr_pred2),
    "r2": r2_score(yr_test, yr_pred2)
}
print("Improved regression:", improved_reg_res)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best cls params: {'model__learning_rate': np.float64(0.14262878834017695), 'model__max_depth': 3, 'model__n_estimators': 213, 'model__subsample': np.float64(0.8019854157170472)}
Best CV ROC-AUC: 0.9843817250544618
Improved classification: {'accuracy': 0.9278, 'precision': 0.9262163607200142, 'recall': 0.9440508628519527, 'f1': 0.9350485786254048, 'roc_auc': np.float64(0.9842844860421156)}
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best reg params: {'model__learning_rate': np.float64(0.06925195035576534), 'model__max_depth': 4, 'model__n_estimators': 251, 'model__subsample': np.float64(0.8737942275278175)}
Best CV -RMSE: -19.877489284584122
Improved regression: {'rmse': 19.972979883216087, 'mae': 15.856033887757498, 'r2': 0.17249698166894978}


## 4. Имплементация градиентного бустинга

Реализованы упрощённые версии градиентного бустинга:
- для регрессии: оптимизация MSE через обучение на остатках;
- для классификации: оптимизация логистической функции потерь через псевдоостатки (y − p).

В качестве базовых моделей используются простые деревья глубины 1 (decision stump), чтобы обучение было быстрым и понятным.


## 5. Утилиты и подготовка данных

In [7]:
rng = np.random.default_rng(RANDOM_STATE)

def to_dense(X):
    return X.toarray() if sparse.issparse(X) else np.asarray(X)

def sigmoid(z):
    z = np.clip(z, -50, 50)
    return 1.0 / (1.0 + np.exp(-z))

def sample_indices(n, max_n=12000, random_state=42):
    if n <= max_n:
        return np.arange(n)
    r = np.random.default_rng(random_state)
    return r.choice(n, size=max_n, replace=False)

# Препроцессинг тот же, что и для sklearn
Xc_tr = prep_cls.fit_transform(Xc_train)
Xc_te = prep_cls.transform(Xc_test)
yc_tr = np.asarray(yc_train).astype(int)
yc_te = np.asarray(yc_test).astype(int)

Xr_tr = prep_reg.fit_transform(Xr_train)
Xr_te = prep_reg.transform(Xr_test)
yr_tr = np.asarray(yr_train).astype(float)
yr_te = np.asarray(yr_test).astype(float)

# Для самописного бустинга удобнее dense
Xc_tr_arr, Xc_te_arr = to_dense(Xc_tr), to_dense(Xc_te)
Xr_tr_arr, Xr_te_arr = to_dense(Xr_tr), to_dense(Xr_te)

print("CLS:", Xc_tr_arr.shape, Xc_te_arr.shape)
print("REG:", Xr_tr_arr.shape, Xr_te_arr.shape)

# Подвыборка для ускорения
idx_c = sample_indices(Xc_tr_arr.shape[0], max_n=12000, random_state=RANDOM_STATE)
idx_r = sample_indices(Xr_tr_arr.shape[0], max_n=12000, random_state=RANDOM_STATE)

Xc_tr_s, yc_tr_s = Xc_tr_arr[idx_c], yc_tr[idx_c]
Xr_tr_s, yr_tr_s = Xr_tr_arr[idx_r], yr_tr[idx_r]

print("Subsample CLS:", Xc_tr_s.shape, "Subsample REG:", Xr_tr_s.shape)


CLS: (40000, 27) (10000, 27)
REG: (15068, 13) (3767, 13)
Subsample CLS: (12000, 27) Subsample REG: (12000, 13)


## 6. Моя базовая модель: Decision Stump (регрессия)

In [8]:
class DecisionStumpRegressor:
    """
    Простое дерево глубины 1: выбирает признак и порог,
    и предсказывает среднее значение в левом/правом узле.
    """
    def __init__(self):
        self.feature_ = None
        self.threshold_ = None
        self.left_value_ = None
        self.right_value_ = None

    def fit(self, X, y, n_thresholds=40):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        n, d = X.shape

        best_loss = np.inf
        best = None

        for f in range(d):
            col = X[:, f]
            uniq = np.unique(col)
            if uniq.size <= 1:
                continue

            # ограничим число порогов
            if uniq.size > n_thresholds:
                thr_list = np.quantile(uniq, np.linspace(0.05, 0.95, n_thresholds))
            else:
                thr_list = (uniq[:-1] + uniq[1:]) / 2.0

            for thr in thr_list:
                mask = col <= thr
                if mask.sum() == 0 or (~mask).sum() == 0:
                    continue

                lv = y[mask].mean()
                rv = y[~mask].mean()
                pred = np.where(mask, lv, rv)
                loss = np.mean((y - pred) ** 2)

                if loss < best_loss:
                    best_loss = loss
                    best = (f, float(thr), float(lv), float(rv))

        if best is None:
            # если ничего не нашли, всегда среднее
            self.feature_ = 0
            self.threshold_ = float(X[:, 0].mean())
            m = float(y.mean())
            self.left_value_ = m
            self.right_value_ = m
        else:
            self.feature_, self.threshold_, self.left_value_, self.right_value_ = best

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        mask = X[:, self.feature_] <= self.threshold_
        return np.where(mask, self.left_value_, self.right_value_)


## 7. Мой Gradient Boosting Regressor

In [9]:
class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=80, learning_rate=0.1, random_state=42):
        self.n_estimators = int(n_estimators)
        self.learning_rate = float(learning_rate)
        self.random_state = int(random_state)
        self.base_ = 0.0
        self.models_ = []

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        self.base_ = float(y.mean())
        pred = np.full_like(y, self.base_, dtype=float)

        self.models_ = []
        for _ in range(self.n_estimators):
            residual = y - pred
            stump = DecisionStumpRegressor().fit(X, residual)
            update = stump.predict(X)
            pred += self.learning_rate * update
            self.models_.append(stump)

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        pred = np.full(X.shape[0], self.base_, dtype=float)
        for stump in self.models_:
            pred += self.learning_rate * stump.predict(X)
        return pred

print("MyGradientBoostingRegressor defined.")


MyGradientBoostingRegressor defined.


## 8. Мой Gradient Boosting Classifier (логистическая потеря)

In [10]:
class MyGradientBoostingClassifier:
    """
    Упрощённый GB для бинарной классификации.
    Моделируем F(x), p = sigmoid(F).
    На каждом шаге обучаем stump на псевдоостатки: r = y - p.
    """
    def __init__(self, n_estimators=80, learning_rate=0.1, random_state=42):
        self.n_estimators = int(n_estimators)
        self.learning_rate = float(learning_rate)
        self.random_state = int(random_state)
        self.base_ = 0.0
        self.models_ = []

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y).astype(float)

        # начальное значение в логитах: log(p/(1-p))
        p0 = np.clip(y.mean(), 1e-6, 1-1e-6)
        self.base_ = float(np.log(p0 / (1 - p0)))

        F = np.full_like(y, self.base_, dtype=float)

        self.models_ = []
        for _ in range(self.n_estimators):
            p = sigmoid(F)
            residual = y - p  # псевдоостаток
            stump = DecisionStumpRegressor().fit(X, residual)
            F += self.learning_rate * stump.predict(X)
            self.models_.append(stump)

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        F = np.full(X.shape[0], self.base_, dtype=float)
        for stump in self.models_:
            F += self.learning_rate * stump.predict(X)
        p1 = sigmoid(F)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X)[:, 1] >= threshold).astype(int)

print("MyGradientBoostingClassifier defined.")


MyGradientBoostingClassifier defined.


## 9. Обучение/оценка самописных моделей (baseline)

In [11]:
#  baseline 
my_gb_cls_base = MyGradientBoostingClassifier(n_estimators=60, learning_rate=0.1, random_state=RANDOM_STATE)
my_gb_cls_base.fit(Xc_tr_s, yc_tr_s)

yc_pred_m = my_gb_cls_base.predict(Xc_te_arr)
yc_proba_m = my_gb_cls_base.predict_proba(Xc_te_arr)[:, 1]

my_baseline_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m),
    "precision": precision_score(yc_te, yc_pred_m, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m),
}
print("MyGB baseline (classification):", my_baseline_cls_res)


my_gb_reg_base = MyGradientBoostingRegressor(n_estimators=60, learning_rate=0.1, random_state=RANDOM_STATE)
my_gb_reg_base.fit(Xr_tr_s, yr_tr_s)

yr_pred_m = my_gb_reg_base.predict(Xr_te_arr)

my_baseline_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m),
    "mae": mean_absolute_error(yr_te, yr_pred_m),
    "r2": r2_score(yr_te, yr_pred_m),
}
print("MyGB baseline (regression):   ", my_baseline_reg_res)


MyGB baseline (classification): {'accuracy': 0.7656, 'precision': 0.726790070311379, 'recall': 0.9200726612170754, 'f1': 0.8120891454224788, 'roc_auc': np.float64(0.8776631821208144)}
MyGB baseline (regression):    {'rmse': 21.4105445882524, 'mae': 16.990754455437628, 'r2': 0.049090283136535295}


## 10. Улучшенный самописный вариант
Идея: больше итераций + меньше learning_rate (обычно стабильнее)

In [12]:
my_gb_cls_imp = MyGradientBoostingClassifier(n_estimators=150, learning_rate=0.05, random_state=RANDOM_STATE)
my_gb_cls_imp.fit(Xc_tr_s, yc_tr_s)

yc_pred_m2 = my_gb_cls_imp.predict(Xc_te_arr)
yc_proba_m2 = my_gb_cls_imp.predict_proba(Xc_te_arr)[:, 1]

my_improved_cls_res = {
    "accuracy": accuracy_score(yc_te, yc_pred_m2),
    "precision": precision_score(yc_te, yc_pred_m2, zero_division=0),
    "recall": recall_score(yc_te, yc_pred_m2, zero_division=0),
    "f1": f1_score(yc_te, yc_pred_m2, zero_division=0),
    "roc_auc": roc_auc_score(yc_te, yc_proba_m2),
}
print("MyGB improved (classification):", my_improved_cls_res)


my_gb_reg_imp = MyGradientBoostingRegressor(n_estimators=150, learning_rate=0.05, random_state=RANDOM_STATE)
my_gb_reg_imp.fit(Xr_tr_s, yr_tr_s)

yr_pred_m2 = my_gb_reg_imp.predict(Xr_te_arr)

my_improved_reg_res = {
    "rmse": root_mean_squared_error(yr_te, yr_pred_m2),
    "mae": mean_absolute_error(yr_te, yr_pred_m2),
    "r2": r2_score(yr_te, yr_pred_m2),
}
print("MyGB improved (regression):   ", my_improved_reg_res)


MyGB improved (classification): {'accuracy': 0.8015, 'precision': 0.7685382972230699, 'recall': 0.9149863760217983, 'f1': 0.8353926527904469, 'roc_auc': np.float64(0.8823811097000502)}
MyGB improved (regression):    {'rmse': 21.3768871125267, 'mae': 16.965951072722532, 'r2': 0.0520776022495919}


## 11. Сравнение результатов и выводы

In [13]:
cls_compare = pd.DataFrame(
    [baseline_cls_res, improved_cls_res, my_baseline_cls_res, my_improved_cls_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

reg_compare = pd.DataFrame(
    [baseline_reg_res, improved_reg_res, my_baseline_reg_res, my_improved_reg_res],
    index=["sk_baseline", "sk_improved", "my_baseline", "my_improved"]
)

display(cls_compare)
display(reg_compare)

,accuracy,precision,recall,f1,roc_auc
sk_baseline,0.9194,0.913135,0.943324,0.927984,0.979734
sk_improved,0.9278,0.926216,0.944051,0.935049,0.984284
my_baseline,0.7656,0.726790,0.920073,0.812089,0.877663
my_improved,0.8015,0.768538,0.914986,0.835393,0.882381


,rmse,mae,r2
sk_baseline,20.705978,16.414578,0.110645
sk_improved,19.972980,15.856034,0.172497
my_baseline,21.410545,16.990754,0.049090
my_improved,21.376887,16.965951,0.052078


## Выводы

В данной лабораторной работе были исследованы модели градиентного бустинга для задач классификации и регрессии, включая библиотечные и самописные реализации.

### Классификация (loan_status)

Градиентный бустинг показал очень высокое качество классификации. Уже бейзлайновая модель `sk_baseline` демонстрирует отличные результаты, а подбор гиперпараметров позволил дополнительно улучшить качество:
- accuracy увеличилась с **0.919 → 0.928**;
- F1-score вырос с **0.928 → 0.935**;
- ROC-AUC достиг значения **0.984**, что говорит о высокой способности модели различать классы.

Самописная реализация градиентного бустинга работает корректно, однако значительно уступает библиотечной версии. Несмотря на улучшение показателей у `my_improved`, значения accuracy и F1-score остаются заметно ниже, что объясняется упрощённой реализацией и отсутствием продвинутых оптимизаций.

### Регрессия (song_popularity)

Для задачи регрессии библиотечная модель градиентного бустинга показала умеренное улучшение качества после настройки гиперпараметров:
- RMSE снизилась с **20.71 → 19.97**;
- коэффициент детерминации **R²** вырос с **0.11 → 0.17**.

Самописные модели регрессии демонстрируют более слабые результаты. Значения R² остаются близкими к нулю, что указывает на низкую точность аппроксимации. Улучшение самописной модели дало лишь незначительный эффект.

### Итог

Градиентный бустинг является одним из наиболее эффективных алгоритмов для задачи классификации и показывает устойчивое улучшение качества при настройке гиперпараметров. В задаче регрессии он превосходит линейные модели и решающие деревья, однако остаётся чувствительным к настройкам. Библиотечные реализации `sklearn` значительно превосходят самописные модели по качеству и стабильности и предпочтительны для практического применения, тогда как самописные реализации носят учебный характер.


# Итоговое сравнение результатов лабораторных работ №1–5

В ходе выполнения лабораторных работ были последовательно исследованы пять алгоритмов машинного обучения: **KNN**, **линейные модели (логистическая и линейная регрессия)**, **решающее дерево**, **случайный лес** и **градиентный бустинг**. Для каждого алгоритма рассматривались задачи классификации и регрессии, а также сравнивались библиотечные и самописные реализации.

### Сравнение алгоритмов в задаче классификации

- **KNN (ЛР1)** показал хорошие результаты после подбора гиперпараметров, однако чувствительность к выбору `k` и высокая вычислительная сложность ограничивают его применение на больших выборках.
- **Логистическая регрессия (ЛР2)** продемонстрировала стабильное, но умеренное качество. Улучшение за счёт настройки регуляризации оказалось незначительным.
- **Решающее дерево (ЛР3)** обеспечило заметный прирост качества по сравнению с линейными моделями, но оказалось склонным к переобучению.
- **Случайный лес (ЛР4)** показал одно из лучших качеств классификации: высокие значения accuracy, F1 и ROC-AUC, при этом модель оказалась устойчивой и стабильной.
- **Градиентный бустинг (ЛР5)** стал лучшим алгоритмом для классификации, обеспечив максимальные значения ROC-AUC и F1-score. Подбор гиперпараметров дал дополнительный, пусть и умеренный, прирост качества.

**Итог по классификации:**  
Наилучшие результаты показали ансамблевые методы — сначала случайный лес, а затем градиентный бустинг, который стал лидером по качеству.

### Сравнение алгоритмов в задаче регрессии

- **KNN-регрессия (ЛР1)** улучшилась после подбора гиперпараметров, но осталась чувствительной к шуму и масштабированию данных.
- **Линейная регрессия (ЛР2)** показала низкие значения R², что указывает на слабую линейную зависимость между признаками и целевой переменной.
- **Решающее дерево (ЛР3)** дало нестабильные результаты и часто показывало низкий или отрицательный R².
- **Случайный лес (ЛР4)** обеспечил наилучшее качество регрессии среди всех алгоритмов, снизив RMSE и повысив R².
- **Градиентный бустинг (ЛР5)** показал улучшение по сравнению с деревом и линейными моделями, однако в данной задаче уступил случайному лесу.

**Итог по регрессии:**  
Наиболее эффективным алгоритмом оказался случайный лес, обеспечивший лучшее соотношение точности и устойчивости.

### Сравнение библиотечных и самописных реализаций

Во всех лабораторных работах самописные модели:
- корректно воспроизводили базовую логику алгоритмов;
- демонстрировали сопоставимые, но более низкие результаты;
- уступали библиотечным реализациям по качеству и стабильности.

Это объясняется отсутствием оптимизаций, более простой реализацией и ограничениями по вычислительным ресурсам.

### Общий вывод

Ансамблевые методы (случайный лес и градиентный бустинг) показали наилучшие результаты как в классификации, так и в регрессии. Подбор гиперпараметров во всех случаях приводил к улучшению качества, особенно для сложных моделей. Библиотечные реализации `sklearn` являются предпочтительными для практического применения, тогда как самописные модели целесообразно рассматривать как учебный инструмент для понимания принципов работы алгоритмов машинного обучения.
